<a href="https://colab.research.google.com/github/aslesani/pgmpy_fork/blob/master/src/default_test/imdb_LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
'''Trains an LSTM model on the IMDB sentiment classification task.
The dataset is actually too small for LSTM to be of any advantage
compared to simpler, much faster methods such as TF-IDF + LogReg.
# Notes
- RNNs are tricky. Choice of batch size is important,
choice of loss and optimizer is critical, etc.
Some configurations won't converge.
- LSTM loss decrease patterns during training can be quite different
from what you see with CNNs/MLPs/etc.
'''
from __future__ import print_function

from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Embedding
from keras.layers import LSTM, SimpleRNN
from keras.datasets import imdb
import numpy as np

In [2]:
import matplotlib.pyplot as plt

def plot_train_val_graph(history):
  loss = history.history['loss']
  val_loss = history.history['val_loss']
  epochs = range(1, len(loss) + 1)
  print('epochs:' , epochs)
  plt.figure()
  plt.plot(epochs, loss, 'bo', label='Training loss')
  plt.plot(epochs, val_loss, 'b', label='Validation loss')
  plt.title('Training and validation loss')
  plt.legend()
  plt.show()


In [3]:
def get_max_len_of_sequences(list_of_sequences):
  lengths = [len(list_of_sequences[i]) for i in range(len(list_of_sequences))]
  return max(lengths) , min(lengths) , lengths

In [4]:
def get_set_of_sensor_events(list_of_sequences):
 
  set_of_sensor_events = set()
  
  for i in range(len(list_of_sequences)):
      set_of_sensor_events = set_of_sensor_events.union(set(list_of_sequences[i]))
  
  return set_of_sensor_events, len(set_of_sensor_events)
  

In [ ]:
def read_data_from_CSV_file(dest_file , data_type ,  has_header = False , return_as_pandas_data_frame = False , remove_date_and_time = False , return_header_separately = False , convert_int_columns_to_int = False):
    '''
    this function is a replacement for read_data_from_PCA_output_file and read_data_from_PCA_digitized_file
    with more capabalities.
    
    Parameters:
    ==========
    dest_file: 
    data_type: type of data that should be read  
    has_header = if the file has header, it is set to True. The header is the first line that starts whit '#' character 
    return_as_pandas_data_frame = if True, the return_value is pandas Dataframe, else numpy ndaaray
    
    convert_int_columns_to_int: if the user want to keep date and time columns, then she should 
                                specify data_type as object and then set convert_int_columns_to_int to True
    
    Returns:
    ========
    return_value: type of it is pandas Dataframe or numpy ndaaray
    
    '''
    header = ""
    with open(dest_file,'r') as dest_f:
        data_iter = csv.reader(dest_f, 
                               delimiter = ',')#quotechar = '"')
    
        if has_header:
            header = next(data_iter)
            header[0] = header[0].split('# ')[1] # remove # from first element
        
        
        data = [data for data in data_iter]
    
    if remove_date_and_time:
        data = np.delete(np.delete(data, -1, 1), -1 , 1)

    return_value= np.asarray(data, dtype = data_type)
    
    if convert_int_columns_to_int:
        rows , cols_to_convert = np.shape(return_value)
        
        if remove_date_and_time == False:
            cols_to_convert -=2
        
        for r in range(rows):
            for c in range(cols_to_convert):
                return_value[r,c] = int(return_value[r,c])
        
    
    if return_as_pandas_data_frame:
        return_value = pd.DataFrame(return_value , columns = header)
        
    if return_header_separately:
        return header , return_value
    
    else:   
        return return_value



In [ ]:
!git clone https://github.com/aslesani/pgmpy_fork.git


In [ ]:
ls

In [ ]:
!git clone https://github.com/aslesani/created_dataset.git

In [ ]:
!rm -r pgmpy_fork  

In [ ]:
cd pgmpy_fork/src/default_test

In [ ]:
cd ..

In [ ]:
ls

In [3]:
def convert_binary_classes_to_zero_and_one(data):
  
  values = sorted(list(set(data)))
  for i in range(len(data)):
    data[i] = values.index(data[i])

    
  return data

In [7]:
def test_convert_binary_classes_to_zero_and_one():
  data = [2,1,1,1,2]
  data = convert_binary_classes_to_zero_and_one(data)
  print(data)

In [ ]:
from keras import backend as K

def mcor(y_true, y_pred):
    #matthews_correlation
    y_pred_pos = K.round(K.clip(y_pred, 0, 1))
    y_pred_neg = 1 - y_pred_pos
 
 
    y_pos = K.round(K.clip(y_true, 0, 1))
    y_neg = 1 - y_pos
 
 
    tp = K.sum(y_pos * y_pred_pos)
    tn = K.sum(y_neg * y_pred_neg)
 
 
    fp = K.sum(y_neg * y_pred_pos)
    fn = K.sum(y_pos * y_pred_neg)
 
 
    numerator = (tp * tn - fp * fn)
    denominator = K.sqrt((tp + fp) * (tp + fn) * (tn + fp) * (tn + fn))
 
 
    return numerator / (denominator + K.epsilon())


def precision(y_true, y_pred):
    """Precision metric.

    Only computes a batch-wise average of precision.

    Computes the precision, a metric for multi-label classification of
    how many selected items are relevant.
    """
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def recall(y_true, y_pred):
    """Recall metric.

    Only computes a batch-wise average of recall.

    Computes the recall, a metric for multi-label classification of
    how many relevant items are selected.
    """
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall


def f1(y_true, y_pred):
    def recall(y_true, y_pred):
        """Recall metric.

        Only computes a batch-wise average of recall.

        Computes the recall, a metric for multi-label classification of
        how many relevant items are selected.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

    def precision(y_true, y_pred):
        """Precision metric.

        Only computes a batch-wise average of precision.

        Computes the precision, a metric for multi-label classification of
        how many selected items are relevant.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision
    precision = precision(y_true, y_pred)
    recall = recall(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))


In [ ]:
!pip install tabulate 

In [4]:
from tabulate import tabulate

#print(tabulate([['Alice', 24], ['Bob', 19]], headers=['algorithm', 'acc']))

def print_list_of_lists(data , headers):
    print(tabulate(data, headers=headers))



In [8]:
def test_print_list_of_lists():
    data = [['Alice', 24], ['Bob', 19]]
    headers=['algorithm', 'acc']
    print_list_of_lists(data , headers)


In [9]:
test_print_list_of_lists()

algorithm      acc
-----------  -----
Alice           24
Bob             19


In [5]:
def imdb_lstm_data_preparation(max_features = 20000, maxlen = 80):
  #max_features = 20000#number_of_events
  # cut texts after this number of words (among top max_features most common words)
  #maxlen = 10#max_seq_len

  print('Loading data...')
  (x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_features)
  print(len(x_train), 'train sequences')
  print(len(x_test), 'test sequences')

  #print('before apply pad_sequence, x_train[0]:' , x_train[0])

  print('Pad sequences (samples x time)')
  x_train = sequence.pad_sequences(x_train, maxlen=maxlen)
  x_test = sequence.pad_sequences(x_test, maxlen=maxlen)
  print('x_train shape:', x_train.shape)
  print('x_test shape:', x_test.shape)
  
  return x_train, x_test, y_train, y_test, max_features, maxlen

In [6]:
from read_write import read_sequence_of_bags_CSV_file_with_activity, read_sequence_based_CSV_file_without_activity, read_data_from_CSV_file
from keras.preprocessing.text import Tokenizer

def my_data_preparation(address_to_read):
  
  list_of_data , list_of_persons = read_sequence_based_CSV_file_without_activity(file_address = address_to_read, has_header = True , separate_data_based_on_persons = False, separate_words= False)
  #sensor_events , number_of_events = get_set_of_sensor_events(sequences)
  
  list_of_persons = convert_binary_classes_to_zero_and_one(list_of_persons)
  
  tokenizer = Tokenizer(num_words = 122, filters='!"#$%&()*+,-./:;<=>?@[\]^`{|}~')
  #list_of_data = [r'salam man', r"'M38_off' , 'M38_on'"]
  tokenizer.fit_on_texts(list_of_data)
  sequences = tokenizer.texts_to_sequences(list_of_data)
  
  max_features = 121#number_of_events
  # cut texts after this number of words (among top max_features most common words)
  maxlen = 10#80#max_seq_len

  #print('before apply pad_sequence, x_train[0]:' , x_train[0])

  #80% of data for train and 20% for test
  train_numbers = int(0.8 * len(sequences))
  x_train, y_train = sequences[0: train_numbers] , list_of_persons[0:train_numbers]
  x_test, y_test = sequences[train_numbers+1:] , list_of_persons[train_numbers+1:]

  print('Pad sequences (samples x time)')
  x_train = sequence.pad_sequences(x_train, maxlen=maxlen)
  x_test = sequence.pad_sequences(x_test, maxlen=maxlen)
  print('x_train shape:', x_train.shape)
  print('x_test shape:', x_test.shape)

  return x_train, x_test, y_train, y_test, max_features, maxlen

In [ ]:
#! cd pgmpy_fork/src/default_test
#!ls


In [ ]:
#!git clone https://github.com/pgmpy/pgmpy 
cd ..
#!ls
#!cd pgmpy/
#pip install -r requirements.txt
#!python setup.py install

In [ ]:
mydata = read_data_from_CSV_file(dest_file = address_to_save , data_type = int ,  has_header = False , return_as_pandas_data_frame = False , remove_date_and_time = True , return_header_separately = False , convert_int_columns_to_int = True)
    

In [ ]:
max_seq_len, min_seq_len , lens = get_max_len_of_sequences(sequences)
print(max_seq_len, min_seq_len)

In [ ]:
print(set(y_train))

In [ ]:
x_train = x_train[0:2500]
y_train = y_train[0:2500]
x_test = x_test[2501:3200]
y_test = y_test[2501:3200]



In [ ]:
y_train = y_train.tolist()
y_test = y_test.tolist()

In [ ]:
print((y_train[0:10]))
print((my_x_train[0:10]))


In [7]:
def create_model_and_apply_on_data(x_train, y_train,x_test, y_test, max_features,embedding_vector_dim = 64, batch_size = 32, epochs = 5, 
                                   loss = 'binary_crossentropy', optimizer='rmsprop', metrics=['accuracy'], plot_train_val_graph = False):
  
  #batch_size = 32

  print('Build model...')
  model = Sequential()
  model.add(Embedding(max_features+1, embedding_vector_dim))
  #model.add(SimpleRNN(64, dropout=0.2, recurrent_dropout=0.2, return_sequences=True))
  #model.add(SimpleRNN(64, dropout=0.2, recurrent_dropout=0.2, return_sequences=True))
  model.add(LSTM(64, dropout=0.2, recurrent_dropout=0.2))
  model.add(Dense(1, activation='sigmoid'))

  model.summary()

  # try using different optimizers and different optimizer configs
  model.compile(loss= loss,
                optimizer=optimizer,
                metrics= metrics)#, mcor,recall, f1])

  print('Train...')
  history = model.fit(x_train, y_train,
            batch_size=batch_size,
            epochs=epochs,
            validation_data=(x_test, y_test))
  score, acc = model.evaluate(x_test, y_test,
                              batch_size=batch_size)
  print('Test score:', score)# i think score is loss value
  print('Test accuracy:', acc)
 
  if plot_train_val_graph:
      plot_train_val_graph(history)
      
  return score, acc, history, len(x_train), len(x_test)

In [ ]:
def make_lstm_without_embedding(x_train, y_train,x_test, y_test, max_features,embedding_vector_dim = 64, batch_size = 32, epochs = 5, 
                                   loss = 'binary_crossentropy', optimizer='rmsprop', metrics=['accuracy'], plot_train_val_graph = False):
  
  #batch_size = 32

  print('Build model...')
  model = Sequential()
  #model.add(Embedding(max_features+1, embedding_vector_dim))
  #model.add(SimpleRNN(64, dropout=0.2, recurrent_dropout=0.2, return_sequences=True))
  #model.add(SimpleRNN(64, dropout=0.2, recurrent_dropout=0.2, return_sequences=True))
  model.add(LSTM(64, dropout=0.2, recurrent_dropout=0.2))
  model.add(Dense(1, activation='sigmoid'))

  model.summary()

  # try using different optimizers and different optimizer configs
  model.compile(loss= loss,
                optimizer=optimizer,
                metrics= metrics)#, mcor,recall, f1])

  print('Train...')
  history = model.fit(x_train, y_train,
            batch_size=batch_size,
            epochs=epochs,
            validation_data=(x_test, y_test))
  score, acc = model.evaluate(x_test, y_test,
                              batch_size=batch_size)
  print('Test score:', score)# i think score is loss value
  print('Test accuracy:', acc)
 
  if plot_train_val_graph:
      plot_train_val_graph(history)
      
  return score, acc, history, len(x_train), len(x_test)

In [8]:
def select_hyperparameters(address_to_read):
    #address_to_read= r"E:/pgmpy/Seq of sensor events_based on activities/based_on_activities.csv"
    #address_to_read = r"E:\pgmpy\Seq of sensor events_no overlap_based on different deltas\delta_{}min.csv"
    #address_to_read = r"E:\pgmpy\Seq of sensor events_based_on_activity_and_no_overlap_delta\delta_{}min.csv"
    results = []
    for delta in list(range(1,16)) + [30,45,60,75,90,100, 120,150, 180,200,240,300,400,500,600,700,800,900,1000]: #:
        x_train, x_test, y_train, y_test, max_features, maxlen = my_data_preparation(address_to_read.format(delta))#imdb_lstm_data_preparation(maxlen=10)
        test_score, test_acc, history, num_of_train_samples, num_of_test_sample = create_model_and_apply_on_data(x_train, y_train,x_test, y_test, max_features)
        results.append([delta, num_of_train_samples, num_of_test_sample, np.mean(history.history['loss']), np.mean(history.history['acc']),history.history['acc'][-1] ,test_score, test_acc])#, history.history
        #print(history.history)
    print_list_of_lists(results, ['delta(min)' ,'#train_samples', '#test_samples', 'train loss ', 'train acc(mean)', 'train_acc(last)', 'val loss ', 'val acc', ])#'history'

In [25]:
#address_to_read= r"E:/pgmpy/Seq of sensor events_based on activities/based_on_activities.csv"
#address_to_read = r"E:\pgmpy\Seq of sensor events_no overlap_based on different deltas\delta_{}min.csv"
address_to_read = r"E:\pgmpy\Seq of sensor events_based_on_activity_and_no_overlap_delta\delta_{}min.csv"
print(address_to_read)    
select_hyperparameters(address_to_read)

E:\pgmpy\Seq of sensor events_based_on_activity_and_no_overlap_delta\delta_{}min.csv
Pad sequences (samples x time)
x_train shape: (13556, 10)
x_test shape: (3389, 10)
Build model...
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_13 (Embedding)     (None, None, 64)          7808      
_________________________________________________________________
lstm_13 (LSTM)               (None, 64)                33024     
_________________________________________________________________
dense_13 (Dense)             (None, 1)                 65        
Total params: 40,897
Trainable params: 40,897
Non-trainable params: 0
_________________________________________________________________
Train...
Train on 13556 samples, validate on 3389 samples
Epoch 1/5
13556/13556 [==============================] - 10s 720us/step - loss: 0.2231 - acc: 0.9045 - val_loss: 0.2625 - val_acc: 0.8799
Epoch 2/5
13556/13556 [

Epoch 5/5
1652/1652 [==============================] - 0s 132us/step
Test score: 0.238481441214
Test accuracy: 0.917675544794
Pad sequences (samples x time)
x_train shape: (6129, 10)
x_test shape: (1532, 10)
Build model...
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_18 (Embedding)     (None, None, 64)          7808      
_________________________________________________________________
lstm_18 (LSTM)               (None, 64)                33024     
_________________________________________________________________
dense_18 (Dense)             (None, 1)                 65        
Total params: 40,897
Trainable params: 40,897
Non-trainable params: 0
_________________________________________________________________
Train...
Train on 6129 samples, validate on 1532 samples
Epoch 1/5
6129/6129 [==============================] - 9s 1ms/step - loss: 0.2594 - acc: 0.8895 - val_loss: 0.2692 - val_a

Epoch 4/5
5033/5033 [==============================] - 3s 599us/step - loss: 0.1468 - acc: 0.9420 - val_loss: 0.3394 - val_acc: 0.8855
Epoch 5/5
1258/1258 [==============================] - 0s 139us/step
Test score: 0.316821835611
Test accuracy: 0.912559618442
Pad sequences (samples x time)
x_train shape: (4856, 10)
x_test shape: (1214, 10)
Build model...
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_23 (Embedding)     (None, None, 64)          7808      
_________________________________________________________________
lstm_23 (LSTM)               (None, 64)                33024     
_________________________________________________________________
dense_23 (Dense)             (None, 1)                 65        
Total params: 40,897
Trainable params: 40,897
Non-trainable params: 0
_________________________________________________________________
Train...
Train on 4856 samples, validate on 

Epoch 3/5
4400/4400 [==============================] - 3s 600us/step - loss: 0.1590 - acc: 0.9366 - val_loss: 0.3787 - val_acc: 0.8682
Epoch 4/5
4400/4400 [==============================] - 3s 598us/step - loss: 0.1512 - acc: 0.9395 - val_loss: 0.3715 - val_acc: 0.8755
Epoch 5/5
1100/1100 [==============================] - 0s 142us/step
Test score: 0.384836446523
Test accuracy: 0.878181818182
Pad sequences (samples x time)
x_train shape: (3572, 10)
x_test shape: (892, 10)
Build model...
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_28 (Embedding)     (None, None, 64)          7808      
_________________________________________________________________
lstm_28 (LSTM)               (None, 64)                33024     
_________________________________________________________________
dense_28 (Dense)             (None, 1)                 65        
Total params: 40,897
Trainable params: 40,897


Epoch 2/5
2899/2899 [==============================] - 2s 729us/step - loss: 0.1801 - acc: 0.9120 - val_loss: 0.9214 - val_acc: 0.7155
Epoch 3/5
2899/2899 [==============================] - 2s 637us/step - loss: 0.1439 - acc: 0.9369 - val_loss: 0.8198 - val_acc: 0.7997
Epoch 4/5
2899/2899 [==============================] - 2s 518us/step - loss: 0.1325 - acc: 0.9407 - val_loss: 0.8327 - val_acc: 0.8066
Epoch 5/5
724/724 [==============================] - 0s 136us/step
Test score: 0.814344453548
Test accuracy: 0.803867402656
Pad sequences (samples x time)
x_train shape: (2864, 10)
x_test shape: (716, 10)
Build model...
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_33 (Embedding)     (None, None, 64)          7808      
_________________________________________________________________
lstm_33 (LSTM)               (None, 64)                33024     
_____________________________________________

2705/2705 [==============================] - 12s 4ms/step - loss: 0.3163 - acc: 0.8677 - val_loss: 1.0954 - val_acc: 0.4527
Epoch 2/5
2705/2705 [==============================] - 1s 529us/step - loss: 0.1800 - acc: 0.9087 - val_loss: 0.7803 - val_acc: 0.7485
Epoch 3/5
2705/2705 [==============================] - 1s 526us/step - loss: 0.1368 - acc: 0.9434 - val_loss: 0.8930 - val_acc: 0.7914
Epoch 4/5
2705/2705 [==============================] - 1s 530us/step - loss: 0.1272 - acc: 0.9431 - val_loss: 1.0236 - val_acc: 0.7530
Epoch 5/5
676/676 [==============================] - 0s 125us/step
Test score: 0.969608951867
Test accuracy: 0.769230769231
Pad sequences (samples x time)
x_train shape: (2676, 10)
x_test shape: (669, 10)
Build model...
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_38 (Embedding)     (None, None, 64)          7808      
_____________________________________________________

c:\users\adele\appdata\local\programs\python\python35\lib\site-packages\keras\callbacks.py:120: UserWarning: Method on_batch_end() is slow compared to the batch update (0.261655). Check your callbacks.
  % delta_t_median)


2608/2608 [==============================] - 30s 12ms/step - loss: 0.3289 - acc: 0.8696 - val_loss: 1.3520 - val_acc: 0.3834
Epoch 2/5
2608/2608 [==============================] - 2s 579us/step - loss: 0.1842 - acc: 0.9053 - val_loss: 0.9975 - val_acc: 0.6104
Epoch 3/5
2608/2608 [==============================] - 2s 594us/step - loss: 0.1394 - acc: 0.9375 - val_loss: 0.8280 - val_acc: 0.7776
Epoch 4/5
2608/2608 [==============================] - 2s 582us/step - loss: 0.1277 - acc: 0.9429 - val_loss: 0.9261 - val_acc: 0.7561
Epoch 5/5
652/652 [==============================] - 0s 129us/step
Test score: 1.0783800885
Test accuracy: 0.680981595458
Pad sequences (samples x time)
x_train shape: (2584, 10)
x_test shape: (646, 10)
Build model...
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_42 (Embedding)     (None, None, 64)          7808      
______________________________________________________

Train on 2575 samples, validate on 643 samples
Epoch 1/5
2575/2575 [==============================] - 18s 7ms/step - loss: 0.3260 - acc: 0.8746 - val_loss: 1.4424 - val_acc: 0.3546
Epoch 2/5
2575/2575 [==============================] - 2s 610us/step - loss: 0.1836 - acc: 0.9091 - val_loss: 0.9559 - val_acc: 0.6065
Epoch 3/5
2575/2575 [==============================] - 2s 622us/step - loss: 0.1396 - acc: 0.9386 - val_loss: 0.9729 - val_acc: 0.7512
Epoch 4/5
2575/2575 [==============================] - 2s 609us/step - loss: 0.1273 - acc: 0.9410 - val_loss: 0.8802 - val_acc: 0.7652
Epoch 5/5
643/643 [==============================] - 0s 132us/step
Test score: 0.950448040829
Test accuracy: 0.754276827464
  delta(min)    #train_samples    #test_samples    train loss     train acc(mean)    train_acc(last epoch)    validation loss     validation acc
------------  ----------------  ---------------  -------------  -----------------  -----------------------  ------------------  ----------------


In [27]:
#address_to_read= r"E:/pgmpy/Seq of sensor events_based on activities/based_on_activities.csv"
address_to_read = r"E:\pgmpy\Seq of sensor events_no overlap_based on different deltas\delta_{}min.csv"
#address_to_read = r"E:\pgmpy\Seq of sensor events_based_on_activity_and_no_overlap_delta\delta_{}min.csv"
print(address_to_read)    
select_hyperparameters(address_to_read)

E:\pgmpy\Seq of sensor events_no overlap_based on different deltas\delta_{}min.csv
Pad sequences (samples x time)
x_train shape: (11619, 10)
x_test shape: (2904, 10)
Build model...
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_47 (Embedding)     (None, None, 64)          7808      
_________________________________________________________________
lstm_47 (LSTM)               (None, 64)                33024     
_________________________________________________________________
dense_47 (Dense)             (None, 1)                 65        
Total params: 40,897
Trainable params: 40,897
Non-trainable params: 0
_________________________________________________________________
Train...
Train on 11619 samples, validate on 2904 samples
Epoch 1/5
11619/11619 [==============================] - 37s 3ms/step - loss: 0.2101 - acc: 0.9148 - val_loss: 0.2440 - val_acc: 0.8867
Epoch 2/5
11619/11619 [====

Epoch 5/5
1108/1108 [==============================] - 0s 147us/step
Test score: 0.163896668969
Test accuracy: 0.92238267148
Pad sequences (samples x time)
x_train shape: (3945, 10)
x_test shape: (986, 10)
Build model...
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_52 (Embedding)     (None, None, 64)          7808      
_________________________________________________________________
lstm_52 (LSTM)               (None, 64)                33024     
_________________________________________________________________
dense_52 (Dense)             (None, 1)                 65        
Total params: 40,897
Trainable params: 40,897
Non-trainable params: 0
_________________________________________________________________
Train...
Train on 3945 samples, validate on 986 samples
Epoch 1/5
3945/3945 [==============================] - 19s 5ms/step - loss: 0.2440 - acc: 0.9280 - val_loss: 0.1445 - val_acc

c:\users\adele\appdata\local\programs\python\python35\lib\site-packages\keras\callbacks.py:120: UserWarning: Method on_batch_end() is slow compared to the batch update (0.121529). Check your callbacks.
  % delta_t_median)


3273/3273 [==============================] - 244s 75ms/step - loss: 0.2550 - acc: 0.9099 - val_loss: 0.1735 - val_acc: 0.9523
Epoch 2/5
3273/3273 [==============================] - 3s 796us/step - loss: 0.1556 - acc: 0.9419 - val_loss: 0.1483 - val_acc: 0.9364
Epoch 3/5
3273/3273 [==============================] - 3s 799us/step - loss: 0.1432 - acc: 0.9435 - val_loss: 0.1600 - val_acc: 0.9315
Epoch 4/5
3273/3273 [==============================] - 3s 781us/step - loss: 0.1398 - acc: 0.9444 - val_loss: 0.1473 - val_acc: 0.9340
Epoch 5/5
818/818 [==============================] - 0s 190us/step
Test score: 0.13204027338
Test accuracy: 0.946210268949
Pad sequences (samples x time)
x_train shape: (2997, 10)
x_test shape: (749, 10)
Build model...
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_55 (Embedding)     (None, None, 64)          7808      
____________________________________________________

Train...
Train on 2349 samples, validate on 587 samples
Epoch 1/5
2349/2349 [==============================] - 28s 12ms/step - loss: 0.2699 - acc: 0.9072 - val_loss: 0.2277 - val_acc: 0.9097
Epoch 2/5
2349/2349 [==============================] - 2s 816us/step - loss: 0.1633 - acc: 0.9413 - val_loss: 0.1402 - val_acc: 0.9438
Epoch 3/5
2349/2349 [==============================] - 2s 803us/step - loss: 0.1489 - acc: 0.9455 - val_loss: 0.1691 - val_acc: 0.9284
Epoch 4/5
2349/2349 [==============================] - 2s 816us/step - loss: 0.1444 - acc: 0.9468 - val_loss: 0.1520 - val_acc: 0.9319
Epoch 5/5
587/587 [==============================] - 0s 218us/step
Test score: 0.168072723381
Test accuracy: 0.943781942078
Pad sequences (samples x time)
x_train shape: (2235, 10)
x_test shape: (558, 10)
Build model...
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_60 (Embedding)     (None, None, 64)       

Train on 741 samples, validate on 185 samples
Epoch 1/5
741/741 [==============================] - 23s 31ms/step - loss: 0.4079 - acc: 0.8435 - val_loss: 0.2290 - val_acc: 0.9459
Epoch 2/5
741/741 [==============================] - 1s 1ms/step - loss: 0.1994 - acc: 0.9204 - val_loss: 0.2190 - val_acc: 0.8973
Epoch 3/5
741/741 [==============================] - 1s 1ms/step - loss: 0.1770 - acc: 0.9339 - val_loss: 0.1683 - val_acc: 0.9297
Epoch 4/5
741/741 [==============================] - 1s 1ms/step - loss: 0.1670 - acc: 0.9312 - val_loss: 0.2046 - val_acc: 0.8757
Epoch 5/5
185/185 [==============================] - 0s 361us/step
Test score: 0.227224701482
Test accuracy: 0.859459458815
Pad sequences (samples x time)
x_train shape: (624, 10)
x_test shape: (156, 10)
Build model...
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_65 (Embedding)     (None, None, 64)          7808      
___________

c:\users\adele\appdata\local\programs\python\python35\lib\site-packages\keras\callbacks.py:120: UserWarning: Method on_batch_end() is slow compared to the batch update (0.205708). Check your callbacks.
  % delta_t_median)


499/499 [==============================] - 554s 1s/step - loss: 0.4862 - acc: 0.8136 - val_loss: 0.3611 - val_acc: 0.9194
Epoch 2/5
499/499 [==============================] - 1s 2ms/step - loss: 0.2328 - acc: 0.9138 - val_loss: 0.2557 - val_acc: 0.9355
Epoch 3/5
499/499 [==============================] - 1s 1ms/step - loss: 0.1912 - acc: 0.9178 - val_loss: 0.2655 - val_acc: 0.8952
Epoch 4/5
499/499 [==============================] - 1s 1ms/step - loss: 0.1743 - acc: 0.9319 - val_loss: 0.2566 - val_acc: 0.8952
Epoch 5/5
124/124 [==============================] - 0s 252us/step
Test score: 0.325258209821
Test accuracy: 0.798387102542
Pad sequences (samples x time)
x_train shape: (435, 10)
x_test shape: (108, 10)
Build model...
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_68 (Embedding)     (None, None, 64)          7808      
_________________________________________________________________
ls

256/256 [==============================] - 26s 101ms/step - loss: 0.6419 - acc: 0.6680 - val_loss: 0.6510 - val_acc: 0.6032
Epoch 2/5
256/256 [==============================] - 0s 1ms/step - loss: 0.4101 - acc: 0.8398 - val_loss: 0.4572 - val_acc: 0.8254
Epoch 3/5
256/256 [==============================] - 0s 1ms/step - loss: 0.3163 - acc: 0.8789 - val_loss: 0.3762 - val_acc: 0.8571
Epoch 4/5
256/256 [==============================] - 0s 1ms/step - loss: 0.2712 - acc: 0.8789 - val_loss: 0.3944 - val_acc: 0.7778
Epoch 5/5
63/63 [==============================] - 0s 562us/step
Test score: 0.453346092077
Test accuracy: 0.730158718805
Pad sequences (samples x time)
x_train shape: (216, 10)
x_test shape: (54, 10)
Build model...
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_73 (Embedding)     (None, None, 64)          7808      
_________________________________________________________________
lst

115/115 [==============================] - 24s 207ms/step - loss: 0.6801 - acc: 0.6609 - val_loss: 0.7090 - val_acc: 0.4643
Epoch 2/5
115/115 [==============================] - 0s 2ms/step - loss: 0.6039 - acc: 0.7913 - val_loss: 0.7188 - val_acc: 0.5000
Epoch 3/5
115/115 [==============================] - 0s 2ms/step - loss: 0.4587 - acc: 0.8174 - val_loss: 0.6271 - val_acc: 0.6071
Epoch 4/5
115/115 [==============================] - 0s 2ms/step - loss: 0.3719 - acc: 0.8783 - val_loss: 0.5396 - val_acc: 0.7143
Epoch 5/5
28/28 [==============================] - 0s 991us/step
Test score: 0.449709683657
Test accuracy: 0.857142865658
Pad sequences (samples x time)
x_train shape: (98, 10)
x_test shape: (24, 10)
Build model...
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_78 (Embedding)     (None, None, 64)          7808      
_________________________________________________________________
lstm

In [ ]:
create_model_and_apply_on_data(x_train, y_train,x_test, y_test, max_features)

In [11]:
address_to_read = r"E:\pgmpy\Seq of Bag of sensor events_based_on_activity_and_no_overlap_delta\delta_{}min.csv"
address_to_read = address_to_read.format(15)
list_of_data , list_of_persons , list_of_activities = read_sequence_of_bags_CSV_file_with_activity(file_address = address_to_read , has_header = True, separate_data_based_on_persons = False)
create_model_and_apply_on_data(x_train=  list_of_data, y_train = list_of_persons,x_test = list_of_data, y_test = list_of_persons, max_features = 100)

In [17]:
print((list_of_data[1]))

[[ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0 25 19 11 12  1  2  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0  0  0  0  1  1  1  0  0  0  0  0  0  0  0  0  0  0  1  0
   1  1 10 10  2  2  1  1  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0]]


In [16]:
address_to_read= r"E:/pgmpy/Seq of sensor events_based on activities/based_on_activities.csv"

x_train, x_test, y_train, y_test, max_features, maxlen = imdb_lstm_data_preparation(maxlen=10)
#my_x_train, my_x_test, my_y_train, my_y_test, my_max_features, my_maxlen = my_data_preparation(address_to_read)
#x_train, x_test, y_train, y_test, max_features, maxlen = my_data_preparation(address_to_read)#imdb_lstm_data_preparation(maxlen=10)
create_model_and_apply_on_data(x_train, y_train,x_test, y_test, max_features,embedding_vector_dim = maxlen)

Loading data...
25000 train sequences
25000 test sequences
Pad sequences (samples x time)
x_train shape: (25000, 10)
x_test shape: (25000, 10)
Build model...
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, None, 10)          200010    
_________________________________________________________________
lstm_1 (LSTM)                (None, 64)                19200     
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 65        
Total params: 219,275
Trainable params: 219,275
Non-trainable params: 0
_________________________________________________________________
Train...
Train on 25000 samples, validate on 25000 samples
Epoch 1/5
25000/25000 [==============================] - 16s 653us/step - loss: 0.6015 - acc: 0.6613 - val_loss: 0.5451 - val_acc: 0.7164
Epoch 2/5
25000/25000 [======================

(0.52888289176940917,
 0.72951999999999995,
 25000,
 25000)

In [ ]:
embeddings = model.layers[0].get_weights()[0]
print(embeddings)

In [ ]:
model.layers[1].

In [ ]:
# `word_to_index` is a mapping (i.e. dict) from words to their index, e.g. `love`: 69
words_embeddings = {w:embeddings[idx] for w, idx in word_to_index.items()}

# now you can use it like this for example
print(words_embeddings['love'])  # possible output: [0.21, 0.56, ..., 0.65, 0.10]


In [ ]:
print(type(score) , type(acc))